<a href="https://colab.research.google.com/github/guzun0325/Data-Analysis/blob/main/%E8%B3%87%E6%96%99%E5%89%8D%E8%99%95%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 資料輸入

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
data = pd.read_csv("1472實驗 20220819_39_2.tsv", sep='\t')

<ipython-input-21-552739ea2fe1>:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("1472實驗 20220819_39_2.tsv", sep='\t')


### 轉換微秒為時間

In [ ]:
from datetime import timedelta

def milliseconds_to_hms(milliseconds: int) -> str:
    """毫秒轉hh:mm:ss.ffff
    """
    time_delta = timedelta(milliseconds=milliseconds)
    total_seconds = time_delta.total_seconds()/1000

    # 1小時3600秒
    seconds_in_hours = timedelta(hours=1).total_seconds()

    # 1分鐘60秒
    seconds_in_minute = timedelta(minutes=1).total_seconds()

    # 小時, 剩餘小時的秒數 = 總秒數/3600秒(1小時=3600秒)
    hours, remainder = divmod(total_seconds, seconds_in_hours)

    # 分鐘, 剩餘秒數 = 剩餘小時的秒數/60秒(1分鐘=60秒)
    minutes, seconds = divmod(remainder, seconds_in_minute)

     # 秒, 毫秒 = 剩餘秒數的整數部分和毫秒部分
    seconds_int = int(seconds)
    milliseconds = int((seconds - seconds_int) * 1000)


    return f'{int(hours):02}:{int(minutes):02}:{seconds_int:02}.{milliseconds:03}'

print(milliseconds_to_hms(1004.115))

00:00:00.001


In [ ]:
time_list = []
for i in range (len(data)):
    time = milliseconds_to_hms(int(data['Recording timestamp'][i]))
    time_list.append(time)
data = pd.concat([pd.DataFrame({'time list':time_list}),data],axis=1)

### 篩選跟點擊相關的事件

In [ ]:
mouse = data['Event'] == 'MouseEvent' #計算那些欄位是Moouse
mouse_data = data[mouse] #取出對應欄位的資料
mouse_data.reset_index(drop=True, inplace=True) #重新調整索引

In [ ]:
def calculate_mouse_intervals(picklist):
    delay_list = []
    for i in range(1, len(picklist)):
        mouse_delay = picklist[i] - picklist[i - 1]  # 計算兩點距離
        ms_dist = mouse_delay / 1000.0  # 微秒轉換為毫秒單位
        delay_list.append(ms_dist)
    return delay_list

mouse_delay = calculate_mouse_intervals(mouse_data['Recording timestamp'])

### 將篩選出的欄位整合

In [ ]:
mouse_form = pd.DataFrame({'time':mouse_data['time list'][:-1],
                           'Recording Timestamp':mouse_data['Recording timestamp'][:-1],
                           'Mouse delay':mouse_delay}) #扣除最後一次按鈕沒有反應時間

In [ ]:
import pandas as pd
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('mouse_click.xlsx', engine='openpyxl')

# Convert the dataframe to an XlsxWriter Excel object.
mouse_form.to_excel(writer, sheet_name='mouse_click')

# Close the Pandas Excel writer and output the Excel file.
writer.close()